In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import qubx
%qubxd

%load_ext autoreload
%autoreload 2
from qubx import lookup
from qubx.core.basics import Instrument, FuturesInfo
from qubx.utils.marketdata.binance import get_binance_symbol_info_for_type
from qubx.utils.pandas import scols

 >  [dev] installed cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# mstruct -> Struct

In [2]:
from qubx.utils import Struct

In [3]:
y = Struct(Struct(x=1), b=Struct(x=1), c = 555)
y

Struct(x=1, b=Struct(x=1), c=555)

In [4]:
y | {'a1': 2}

Struct(x=1, b=Struct(x=1), c=555, a1=2)

In [5]:
y.ast = 1

In [6]:
dict(s=1)

{'s': 1}

In [7]:
y

Struct(x=1, b=Struct(x=1), c=555, a1=2, ast=1)

# Market info and symbols

In [8]:
s0 = lookup['BINANCE:ETH.*']
s1 = lookup['DUKAS:EURGBP']

In [9]:
(
    lookup.find_aux_instrument_for(s0[0], 'USDT'), 
    lookup.find_aux_instrument_for(s0[1], 'USDT'), 
    lookup.find_aux_instrument_for(s1[0], 'USD'),
)

(Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', quote='USDT', margin_symbol=None, min_tick=0.01, min_size_step=1e-05, min_size=1e-05, futures_info=None, _aux_instrument=None),
 None,
 Instrument(symbol='GBPUSD', market_type='FX', exchange='DUKAS', base='GBP', quote='USD', margin_symbol='USD', min_tick=1e-05, min_size_step=1, min_size=1000, futures_info=None, _aux_instrument=None))

: 

# Fees

In [6]:
lookup.fees['bitmex_tierb_xbt']

[('bitmex_tierb_xbt', (0.02, 0.075))]

In [7]:
lookup.find_fees('binance', 'um_vip0_usdt')

<binance_um_vip0_usdt: 0.0200 / 0.0500>

In [17]:
lookup.fees

name:			(maker, taker)
binance_spot_vip0_usdt   : (0.1, 0.1)
binance_spot_vip1_usdt   : (0.09, 0.1)
binance_spot_vip2_usdt   : (0.08, 0.1)
binance_spot_vip3_usdt   : (0.042, 0.06)
binance_spot_vip4_usdt   : (0.042, 0.054)
binance_spot_vip5_usdt   : (0.036, 0.048)
binance_spot_vip6_usdt   : (0.03, 0.042)
binance_spot_vip7_usdt   : (0.024, 0.036)
binance_spot_vip8_usdt   : (0.018, 0.03)
binance_spot_vip9_usdt   : (0.012, 0.024)
binance_um_vip0_usdt     : (0.02, 0.05)
binance_um_vip1_usdt     : (0.016, 0.04)
binance_um_vip2_usdt     : (0.014, 0.035)
binance_um_vip3_usdt     : (0.012, 0.032)
binance_um_vip4_usdt     : (0.01, 0.03)
binance_um_vip5_usdt     : (0.008, 0.027)
binance_um_vip6_usdt     : (0.006, 0.025)
binance_um_vip7_usdt     : (0.004, 0.022)
binance_um_vip8_usdt     : (0.002, 0.02)
binance_um_vip9_usdt     : (0.0, 0.017)
bitmex_tierb_xbt         : (0.02, 0.075)
bitmex_tierb_usdt        : (-0.015, 0.075)
bitmex_tieri_xbt         : (0.01, 0.05)
bitmex_tieri_usdt        : (-0.015